# Tutorial: Serializing LLM Pipelines

- **Level**: Beginner
- **Time to complete**: 10 minutes
- **Components Used**: [`HuggingFaceLocalChatGenerator`](https://docs.haystack.deepset.ai/docs/huggingfacelocalchatgenerator), [`ChatPromptBuilder`](https://docs.haystack.deepset.ai/docs/chatpromptbuilder)
- **Prerequisites**: None
- **Goal**: After completing this tutorial, you'll understand how to serialize and deserialize between YAML and Python code.

## Overview

**📚 Useful Documentation:** [Serialization](https://docs.haystack.deepset.ai/docs/serialization)

Serialization means converting a pipeline to a format that you can save on your disk and load later. It's especially useful because a serialized pipeline can be saved on disk or a database, get sent over a network and more. 

Although it's possible to serialize into other formats too, Haystack supports YAML out of the box to make it easy for humans to make changes without the need to go back and forth with Python code. In this tutorial, we will create a very simple pipeline in Python code, serialize it into YAML, make changes to it, and deserialize it back into a Haystack `Pipeline`.

## Installing Haystack

Install Haystack with `pip`:

In [ ]:
%%bash

pip install haystack-ai

## Creating a Simple Pipeline

First, let's create a very simple pipeline that expects a `topic` from the user, and generates a summary about the topic with `Qwen/Qwen2.5-1.5B-Instruct`. Feel free to modify the pipeline as you wish. Note that in this pipeline we are using a local model that we're getting from Hugging Face. We're using a relatively small, open-source LLM.

In [3]:
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import HuggingFaceLocalChatGenerator

template = [
    ChatMessage.from_user(
        """
Please create a summary about the following topic:
{{ topic }}
"""
    )
]

builder = ChatPromptBuilder(template=template)
llm = HuggingFaceLocalChatGenerator(model="Qwen/Qwen2.5-1.5B-Instruct", generation_kwargs={"max_new_tokens": 150})

pipeline = Pipeline()
pipeline.add_component(name="builder", instance=builder)
pipeline.add_component(name="llm", instance=llm)

pipeline.connect("builder.prompt", "llm.messages")

🚅 Components
  - builder: ChatPromptBuilder
  - llm: HuggingFaceLocalChatGenerator
🛤️ Connections
  - builder.prompt -> llm.messages (List[ChatMessage])

In [4]:
topic = "Climate change"
result = pipeline.run(data={"builder": {"topic": topic}})
print(result["llm"]["replies"][0].text)

Climate change is a global issue that has been on the rise in recent years due to various factors such as human activities, natural disasters, and climate variability. The impacts of climate change can be seen in extreme weather events, rising sea levels, melting ice caps, and changing ecosystems.
There are different types of climate change, including global warming, local climate change, and regional climate change. Global warming refers to changes in the Earth's temperature caused by greenhouse gas emissions from human activities, while local climate change refers to changes in the climate of specific areas or regions.
To address climate change, there needs to be a concerted effort from governments, businesses, and individuals around the world to reduce greenhouse gas emissions, protect natural habitats, and invest in renewable energy sources


## Serialize the Pipeline to YAML

Out of the box, Haystack supports YAML. Use `dumps()` to convert the pipeline to YAML:

In [16]:
yaml_pipeline = pipeline.dumps()

print(yaml_pipeline)

components:
  builder:
    init_parameters:
      required_variables: null
      template:
      - _content:
        - text: '

          Please create a summary about the following topic:

          {{ topic }}

          '
        _meta: {}
        _name: null
        _role: user
      variables: null
    type: haystack.components.builders.chat_prompt_builder.ChatPromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 150
        stop_sequences: []
      huggingface_pipeline_kwargs:
        device: cpu
        model: Qwen/Qwen2.5-1.5B-Instruct
        task: text-generation
      streaming_callback: null
      token:
        env_vars:
        - HF_API_TOKEN
        - HF_TOKEN
        strict: false
        type: env_var
    type: haystack.components.generators.chat.hugging_face_local.HuggingFaceLocalChatGenerator
connections:
- receiver: llm.messages
  sender: builder.prompt
max_runs_per_component: 100
metadata: {}



You should get a pipeline YAML that looks like the following:

```yaml
components:
  builder:
    init_parameters:
      required_variables: null
      template:
      - _content:
        - text: '

            Please create a summary about the following topic:

            {{ topic }}

            '
        _meta: {}
        _name: null
        _role: user
      variables: null
    type: haystack.components.builders.chat_prompt_builder.ChatPromptBuilder
  llm:
    init_parameters:
      init_parameters:
      generation_kwargs:
        max_new_tokens: 150
        stop_sequences: []
      huggingface_pipeline_kwargs:
        device: cpu
        model: Qwen/Qwen2.5-1.5B-Instruct
        task: text-generation
      streaming_callback: null
      token:
        env_vars:
        - HF_API_TOKEN
        - HF_TOKEN
        strict: false
        type: env_var
    type: haystack.components.generators.chat.hugging_face_local.HuggingFaceLocalChatGenerator
connections:
- receiver: llm.messages
  sender: builder.prompt
max_runs_per_component: 100
metadata: {}

```

## Editing a Pipeline in YAML

Let's see how we can make changes to serialized pipelines. For example, below, let's modify the `ChatPromptBuilder`'s template to translate provided `sentence` to French:

In [5]:
yaml_pipeline = """
components:
  builder:
    init_parameters:
      template:
      - _content:
        - text: 'Please translate the following to French: \n{{ sentence }}\n'
        _meta: {}
        _name: null
        _role: user
      variables: null
    type: haystack.components.builders.chat_prompt_builder.ChatPromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 150
        stop_sequences: []
      huggingface_pipeline_kwargs:
        device: cpu
        model: Qwen/Qwen2.5-1.5B-Instruct
        task: text-generation
      streaming_callback: null
      chat_template : "{% for message in messages %}{% if message['role'] == 'user' %}{{ ' ' }}{% endif %}{{ message['content'] }}{% if not loop.last %}{{ '  ' }}{% endif %}{% endfor %}{{ eos_token }}"
      token:
        env_vars:
        - HF_API_TOKEN
        - HF_TOKEN
        strict: false
        type: env_var
    type: haystack.components.generators.chat.hugging_face_local.HuggingFaceLocalChatGenerator
connections:
- receiver: llm.messages
  sender: builder.prompt
max_runs_per_component: 100
metadata: {}
"""

## Deseriazling a YAML Pipeline back to Python

You can deserialize a pipeline by calling `loads()`. Below, we're deserializing our edited `yaml_pipeline`:

In [6]:
from haystack import Pipeline

new_pipeline = Pipeline.loads(yaml_pipeline)

Now we can run the new pipeline we defined in YAML. We had changed it so that the `ChatPromptBuilder` expects a `sentence` and translates the sentence to French:

In [7]:
new_pipeline.run(data={"builder": {"sentence": "I love capybaras"}})

{'llm': {'replies': [ChatMessage(content='J'aime les capybaras', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'finish_reason': 'stop', 'index': 0, 'model': 'Qwen/Qwen2.5-1.5B-Instruct', 'usage': {'completion_tokens': 13, 'prompt_tokens': 16, 'total_tokens': 29}})]}}

## What's next

🎉 Congratulations! You've serialized a pipeline into YAML, edited it and ran it again!

If you liked this tutorial, you may also enjoy:
-  [Creating Your First QA Pipeline with Retrieval-Augmentation](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline)

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates). Thanks for reading!